# Load data in DB

There are 4 tables to fill (in order):
- importers
- importers_addresses
- products
- importers_products
- distances

--------------------------------

In [1]:
import pandas as pd
print("pandas version:", pd.__version__)
pd.set_option('precision', 6)

pandas version: 0.23.4


In [2]:
from secrets.secrets_db import hostname, user, password, database  # everything in subfolder "secrets" is .gitignore

In [3]:
!pip install mysql

In [69]:
from sqlalchemy import create_engine
def start_con():
    engine_string = "mysql://{}:{}@{}/{}".format(user, password, hostname, database)
    engine = create_engine(engine_string)
    return engine.connect()

In [70]:
con = start_con()

In [ ]:
# con.close()

## importers

In [5]:
importers = pd.read_pickle("data/importer_ids.pkl")
importers.shape

(30530,)

In [6]:
importers.head(3)

0    #1 ANTIQUE WAREHOUSE FURNISHINGS LTD.
1                               #1 APPAREL
2             0 WASTE 2 ENERGY CANADA LTD.
dtype: object

In [7]:
importers = importers.reset_index()

In [8]:
importers.columns = ["importer_id", "importer_name"]

In [9]:
importers.head()

,importer_id,importer_name
0,0,#1 ANTIQUE WAREHOUSE FURNISHINGS LTD.
1,1,#1 APPAREL
2,2,0 WASTE 2 ENERGY CANADA LTD.
3,3,0021567 B.C. LTD.
4,4,0187993 MANITOBA LIMITED


In [10]:
# what's presently in the DB table?
pd.read_sql("select * from importers", con)

,importer_id,importer_name


In [11]:
%%time
importers.to_sql(name='importers', con=con, if_exists = 'append', index=False)

Wall time: 1.93 s


In [12]:
pd.read_sql("select * from importers limit 10", con)

,importer_id,importer_name
0,0,#1 ANTIQUE WAREHOUSE FURNISHINGS LTD.
1,1,#1 APPAREL
2,2,0 WASTE 2 ENERGY CANADA LTD.
3,3,0021567 B.C. LTD.
4,4,0187993 MANITOBA LIMITED
5,5,0250909 B.C. LTD.
6,6,0266530 B.C. LTD.
7,7,0415393 B.C. LTD.
8,8,0501 FASHIONS INC.
9,9,051865 NB LTD.


In [36]:
pd.read_sql("select count(1) from importers", con)

,count(1)
0,30530


In [ ]:
# con.close()

## importer addresses

Why put addresses in a different table than <code>importers</code>? Because some importers have more than one address.

In [16]:
impprod = pd.read_pickle("data/importers_products.pkl")
impprod.shape

(108900, 3)

In [17]:
impprod = impprod.reset_index()
impprod.drop("hs6", axis=1, inplace=True)
impprod.head(3)

,importer,city,province,postalcode
0,9072-9435 QUEBEC INC.,Saint-Marc-sur-Richelieu,Quebec,J0L 2E0
1,CAMELOT STABLES,Delta,British Columbia,V4K 1S9
2,COUNTRY LANE FARM,Delta,British Columbia,V4K 3N2


In [24]:
impprod.sort_values(by=list(impprod.columns), axis=0, inplace=True)

In [26]:
impprod.drop_duplicates(inplace=True)

In [27]:
impprod.shape

(30549, 4)

In [30]:
merged = impprod.merge(importers, left_on="importer", right_on="importer_name", how="inner")

In [37]:
merged[['importer_id', 'city', 'province', 'postalcode']]\
.to_sql(name='importers_addresses', con=con, if_exists = 'append', index=False)

In [42]:
pd.read_sql("select count(1) from importers_addresses", con)

,count(1)
0,30549


## products

In [43]:
products = pd.read_pickle("data/products.pkl")
products.shape

(5121, 1)

In [45]:
products.reset_index(inplace=True)
products.columns=["product_id", "product_name"]

In [46]:
products.head()

,product_id,product_name
0,010121,"Horses, Pure-Bred Breeding Animals"
1,010129,"Horses, Other"
2,010130,"Horses, Asses"
3,010190,"Horses, Asses, Mules And Hinnies, Live - Not P..."
4,010221,"Cattle, Pure-Bred Breeding Animals"


In [47]:
products.dtypes

product_id      object
product_name    object
dtype: object

In [50]:
products.to_sql(name='products', con=con, if_exists = 'append', index=False)

In [51]:
pd.read_sql("select count(1) from products", con)

,count(1)
0,5121


## importers_products

In [55]:
impprod = pd.read_pickle("data/importers_products.pkl")

In [56]:
impprod.head()

,,city,province,postalcode
importer,hs6,,,
9072-9435 QUEBEC INC.,010121,Saint-Marc-sur-Richelieu,Quebec,J0L 2E0
CAMELOT STABLES,010121,Delta,British Columbia,V4K 1S9
COUNTRY LANE FARM,010121,Delta,British Columbia,V4K 3N2
FOXTRAIL FARMS,010121,Okotoks,Alberta,T1S 1A1
HOERDT'S HOT SHOT SERVICE LTD.,010121,Beaumont,Alberta,T4X 1H9


In [57]:
impprod = impprod.reset_index()\
            .merge(importers, left_on="importer", right_on="importer_name", how="inner")

In [59]:
impprod = impprod[["importer_id", "hs6"]]
impprod.columns = ["importer_id", "product_id"]

In [80]:
impprod = impprod.drop_duplicates()

In [60]:
impprod.dtypes

importer_id     int64
product_id     object
dtype: object

In [95]:
# products which appear in importers_products, but for which there is no product name 
# (hence the product is not in the products table)

prods_in_db = set(products.product_id)
impprod.loc[~impprod.product_id.isin(prods_in_db), "product_id"].value_counts()

854020    11
Name: product_id, dtype: int64

In [97]:
prod_no_name = set(impprod.loc[~impprod.product_id.isin(prods_in_db), "product_id"].unique())
prod_no_name

{'854020'}

We'll remove product id "854020"

In [98]:
impprod.shape

(108887, 2)

In [99]:
impprod = impprod[~impprod.product_id.isin(prod_no_name)]
impprod.shape

(108876, 2)

In [100]:
con = start_con()

In [101]:
pd.read_sql("select * from importers_products limit 10", con)

,importer_id,product_id


In [102]:
impprod.to_sql(name='importers_products', con=con, if_exists = 'append', index=False)

In [103]:
pd.read_sql("select count(1) from importers_products", con)

,count(1)
0,108876


In [104]:
con.close()

## distances

### import data

In [106]:
%%time
dist_matrix = pd.read_pickle("data/dist_matrix_with_imp_ids_sparse.pkl")
dist_matrix.shape

Wall time: 2.33 s


In [107]:
%%time
dist_matrix.iloc[500:505, 500:505]

Wall time: 10.8 s


,500,501,502,503,504
500,0.0,1.0,1.0,1.0,1.0
501,1.0,0.0,1.0,1.0,1.0
502,1.0,1.0,0.0,1.0,1.0
503,1.0,1.0,1.0,0.0,1.0
504,1.0,1.0,1.0,1.0,0.0


#### helper functions

In [108]:
def similar_importers(imp_id, n=None):
    '''
    imp_ind: importer ID (int)
    n: number of values to return (int)
    '''
    if n is not None and n<=0:
        raise ValueError("n must be positive")
    
    similar = dist_matrix.iloc[(dist_matrix.index!=imp_id),imp_id]\
        .to_dense()\
        .sort_values(ascending=True)
    
    similar = similar[similar<1]
    
    if n is None:
        return similar
    else:
        n_to_return = min(n, similar.size)
        return similar[0:n_to_return]

In [128]:
bnc = 3262 #banque nationale du canada
asus = 2703
rolex = 23501

In [132]:
%%time
temp = similar_importers(asus, n=5)
print(temp.size)

5
Wall time: 228 ms


In [133]:
%%time
temp = similar_importers(asus)
print(temp.size)

321
Wall time: 256 ms


----------------------
testing if look-alikes make sense

In [140]:
ids = pd.read_pickle("data/importer_ids.pkl")
ids.shape

(30530,)

In [141]:
def imp_id_to_name(imp_id):
    return ids[imp_id]

In [142]:
def similar_importers_names(imp_id, n=None):
    similar = similar_importers(imp_id, n)
    similar.index = [imp_id_to_name(i) for i in similar.index]
    return similar

In [143]:
%%time
similar_importers_names(bnc, n=7)

Wall time: 18 ms


EXCHANGE BANK OF CANADA / BANQUE DE CHANGE DU CANADA    0.0
TRAVELEX CANADA LIMITED/ TRAVELEX CANADA LIMITEE        0.0
BANK OF AMERICA CANADA                                  0.0
Name: 3262, dtype: float64

In [145]:
similar_importers_names(rolex, n=10)

THE SWATCH GROUP CANADA LTD/GROUPE SWATCH CANADA LTEE    0.645161
BREITLING CANADA INC.                                    0.700000
RICHEMONT CANADA, INC.                                   0.800000
ALPINE INTERNATIONAL                                     0.809524
FOSSIL CANADA INC                                        0.815789
TENSE ENTERPRISES INC.                                   0.850000
LVMH WATCH & JEWELRY CANADA LTD.                         0.857143
PANDORA JEWELRY LTD.                                     0.857143
THE EDWIN M KNOWLES CHINA COMPY                          0.863636
H. & W. PERRIN COMPANY, LIMITED                          0.863636
Name: 23501, dtype: float64

----------------------
back to inserting data in DB

In [158]:
# def insert_in_distances(this_row):
#     df = similar_importers(this_row, 100)
#     df = df.reset_index()
#     df.columns = ["similar_importer_id", "distance"]
#     df["importer_id"] = this_row
#     try:
#         df.to_sql(name='distances', con=con, if_exists = 'append', index=False)
#     except:
#         con = start_con()
#         df.to_sql(name='distances', con=con, if_exists = 'append', index=False)
        

In [241]:
import time

In [251]:
def insert_in_distances(rows):
    time.sleep(0.2)
    con = start_con()
    main_df = pd.DataFrame(columns=["similar_importer_id", "distance", "importer_id"])
    for this_row in rows:
        df = similar_importers(this_row, 100)
        df = df.reset_index()
        df.columns = ["similar_importer_id", "distance"]
        df["importer_id"] = this_row
        main_df = main_df.append(df)
    
    main_df.to_sql(name='distances', con=con, if_exists = 'append', index=False)
    con.close()

In [244]:
con.close()

In [245]:
import numpy as np

In [246]:
n_importers = dist_matrix.shape[0]
n_importers

30530

In [252]:
import datetime

In [ ]:
%%time
n_splits = 300
splits = np.array_split(range(n_importers), n_splits)
start = datetime.datetime.now()
for i,split in enumerate(splits):
    if i%25==0:
        min_since_start = (datetime.datetime.now() - start).seconds/60
        print("At {} of {} (Minutes since start: {})".format(str(i), str(n_splits), str(min_since_start)))
    insert_in_distances(split)

At 0 of 300 (Minutes since start: 0.0)
At 25 of 300 (Minutes since start: 7.05)


----------------

In [257]:
con.execute("delete from distances;")

In [255]:
con = start_con()

In [258]:
pd.read_sql("select count(1) from distances", con)

,count(1)
0,0


In [216]:
import datetime

In [218]:
a = datetime.datetime.now()

In [237]:
(datetime.datetime.now() - a).seconds /60

0.85